In [1]:
# Importing the modules

import random
import pandas as pd
import random


pd.set_option("display.max_rows", 16)

#### Functions

In [ ]:
def path_selection_function(roulette):
    """
    Selects a path from roulette wheel selection based on probabilities.

    Args:
        roulette (list) : A list of probabilities associated with potential parents.

    Returns:
        int : The index of the selected path in the input list.
    """
    
    value = random.random()
    for i, element in enumerate(roulette):
        if element > value:
            return i

In [ ]:
def selecao_destino_function(tabela_probabilidade, nao_visitadas, rota):

    

    # Seleciona a Cidade atual 
    m1 = tabela_probabilidade['Rotas_Out'] == rota[-1]

    # Seleciona Possiveis Cidades Destino
    m2 = tabela_probabilidade['Rotas_In'].isin(nao_visitadas)

    # Cria um DF temporarios com possiveis destinos
    df_temp = tabela_probabilidade[m1 & m2].copy()
    df_temp.reset_index(drop=True, inplace = True)

    # Cria um campo de probabildiade da cidade ser visitada
    divisor = df_temp['Tal_x_Feromonio'].sum()
    df_temp['P_xy'] =  df_temp['Tal_x_Feromonio'] / divisor 

    # Cria uma Roleta para sorteio
    roleta = df_temp['P_xy'].copy()
    for psx, ps in enumerate(roleta[1:]):
        roleta[psx+1] = ps + roleta[psx]

    return df_temp.loc[path_selection_function(roleta), 'Rotas_In']

In [ ]:
def resultado_function(rota, df, change_duration, service_duration, cost, velocity,truck_capacity, sellers_capacity):
    """
    """
    truck = 0
    seller = 0
    profit = 0

    for rdx, r in enumerate(rota[:-1]):

        nn = rota[rdx+1]
        truck_aux = truck + df.loc[nn, 'Cyn']
        seller_aux = seller + (df.loc[nn, 'Cyn'] * change_duration) + service_duration + (cost[(r, nn)] / velocity) + (cost[(nn, 0)] / velocity)
        
        if (truck_aux <= truck_capacity) & (seller_aux <= sellers_capacity):
            seller += (df.loc[nn, 'Cyn'] * change_duration) + service_duration + (cost[(r, nn)] / velocity)
            truck += df.loc[nn, 'Cyn']
            profit += df.loc[nn, 'Val']

    seller += (cost[(nn, 0)] / velocity)


    return seller, truck, profit
    

In [ ]:
def viabilidade_rota_function(rota, df, change_duration, service_duration, cost, velocity,truck_capacity, sellers_capacity):
    """
    """
    truck = 0
    seller = 0

    for rdx, r in enumerate(rota[:-1]):

        nn = rota[rdx+1]
        truck_aux = truck + df.loc[nn, 'Cyn']
        seller_aux = seller + (df.loc[nn, 'Cyn'] * change_duration) + service_duration + (cost[(r, nn)] / velocity) + (cost[(nn, 0)] / velocity)
        
        if (truck_aux <= truck_capacity) & (seller_aux <= sellers_capacity):
            seller += (df.loc[nn, 'Cyn'] * change_duration) + service_duration + (cost[(r, nn)] / velocity)
            truck += df.loc[nn, 'Cyn']
        else:
            return True

    return False
    

In [ ]:
def atualizando_feromonios_function(tabela_probabilidade, coef_evaporacao, formigas, const_atualizacao_feromonio):
    """
    """
    
    atualizacao_feromonio = tabela_probabilidade[['Rotas_Out', 'Rotas_In']].copy()
    atualizacao_feromonio['(1 - CE) * Txy'] = (1 - coef_evaporacao) * tabela_probabilidade['Feromonio']
    colunas = ['(1 - CE) * Txy']

    for f in formigas:
        # Calcula a Concentração de Feromonio Deixada Pela Formiga
        concentracao_feromonio = const_atualizacao_feromonio / (formigas[f]['Lucro'])
        # Cria uma coluna da Formiga para ela deixar seur feromonios
        atualizacao_feromonio[f] = 0.0

        colunas.append(f)
        rota = formigas[f]['rota']

        for rdx, r in enumerate(rota):
            try:
                m1 = (atualizacao_feromonio['Rotas_Out'] == r) | (atualizacao_feromonio['Rotas_Out'] == rota[rdx+1])
                m2 = (atualizacao_feromonio['Rotas_In'] == rota[rdx+1]) | (atualizacao_feromonio['Rotas_In'] == r)
                atualizacao_feromonio.loc[m1 & m2, f] = concentracao_feromonio
            except:
                pass

    atualizacao_feromonio['Total'] = atualizacao_feromonio[colunas].sum(axis = 1)
    tabela_probabilidade = tabela_probabilidade.merge(  atualizacao_feromonio[['Rotas_Out', 'Rotas_In', 'Total']], 
                                                        on = ['Rotas_Out', 'Rotas_In'], 
                                                        how = 'inner')
    tabela_probabilidade.drop(columns = 'Feromonio', inplace = True)
    tabela_probabilidade.rename(columns = {'Total' : 'Feromonio'}, inplace = True)
    tabela_probabilidade['Tal_x_Feromonio'] = tabela_probabilidade['Tal'] * tabela_probabilidade['Feromonio']

    return tabela_probabilidade

In [45]:
def aco_execution_function(iteracoes, total_formigas, total_customers,tabela_probabilidade, df, change_duration, service_duration, cost, velocity, truck_capacity,
                  sellers_capacity, best, coef_evaporacao, const_atualizacao_feromonio, sellers = False, rotas = []):
    """
    """

    for i in range(iteracoes):
        
        formigas = {}
        for tf in range(total_formigas):
            
            # Lista de Clietes não visitados
            nao_visitadas = list(range(1, total_customers))

            if sellers:
                for v in rotas:
                    nao_visitadas.remove(v)
            # Criterio de finalizacao de rota
            exit = True
            # Criando Rota da Formiga
            rota = [0]
            
            
            while exit:
                
                # Seleciona o Possivel Destino
                destino = selecao_destino_function(tabela_probabilidade, nao_visitadas, rota)
                rota.append(destino)
                nao_visitadas.remove(destino)
                
                
                # Verificacao destino viavel
                if viabilidade_rota_function(rota, df, change_duration, service_duration, cost, velocity,truck_capacity, sellers_capacity): 
                    
                    rota.remove(destino)
                    rota.append(0)
                    exit = False
                
                elif len(nao_visitadas) == 0:
                    rota.append(0)
                    exit = False
                else:
                    pass
                

                    
            seller, truck, profit = resultado_function(rota, df, change_duration, service_duration, cost, velocity,truck_capacity, sellers_capacity)
            formigas[f'F{tf}'] = {  'rota' : rota,
                                    'tempo' : seller,
                                    'carga' : truck,
                                    'Lucro' : profit}
            
            if profit > best['Lucro']:
                best = {'rota' : rota,
                        'tempo' : seller,
                        'carga' : truck,
                        'Lucro' : profit}
        
        tabela_probabilidade = atualizando_feromonios_function(tabela_probabilidade, coef_evaporacao, formigas, const_atualizacao_feromonio)
        
    return best, tabela_probabilidade

#### Parameters

In [ ]:
# Number of Customers 
total_customers = 5

# Number of Sellers / Trucks
total_sellers = 1

# Seller capacity
sellers_capacity = 28800 # s (8h/day)

# Truck capacity
truck_capacity = 60 # unit (High Pressure Cylinders)

# Truck Speed
velocity = 13.9 # m/s (Normal speed of a Truck (50 km/h))

# Dutarion Service
change_duration = 300 # 5 minutes on average to change a cylinder
service_duration = 600 # 5 minutes per stop

# ID Source
warehouse = 0
 
# the bigM
bigM = total_customers + 1

# Bidirectional Graph: G(V,A)
# (C[i], C[j]) : Cost

df = pd.DataFrame()

# C0 = Warehouse
customers = ['C0']
long = [0]
lat = [0]
cyn = [0]
val = [0]

X, Y  = 2600, 2600
# Generate values randomly to create the scenario
for c in range(1, total_customers+1):
    customers.append(f'C{c}')
    long.append(0 + X * random.random())
    lat.append(0 + Y * random.random())
    cyn.append( random.choice(range(1, 10)) )
    val.append( random.choice(range(80, 1500)) * cyn[-1])

df['Customers'] = customers
df['Log'] = long
df['Lat'] = lat  
df['Cyn'] = cyn
df['Val'] = val


# Calculate the euclidian distance between each point on the map
cost = {}
for i in df.index:
    for j in df.index:

        x = df.loc[i, 'Log'] - df.loc[j, 'Log']
        y = df.loc[i, 'Lat'] - df.loc[j, 'Lat']
        cost[(i, j)] = (x**2 + y**2)**(0.5)



In [ ]:
# ACO parameters
# ---------------------

alfa = 1
beta = 1
coef_evaporacao = 0.01 
feromonio_inicial = 0.1
const_atualizacao_feromonio = 10

total_formigas = 10
iteracoes = 50


best = {    'rota' : [0],
            'tempo' : 0,
            'carga' : 0,
            'Lucro' : -1}

In [ ]:
tabela_probabilidade = pd.DataFrame( columns = ['Rotas_Out', 'Rotas_In', 'Distancia'])

Rotas_Out = []
Rotas_In = []
Distancia = []

for i in df.index:
    for j in df.index:
        if j != i:
            Rotas_Out.append(i)
            Rotas_In.append(j)
            Distancia.append(cost[(i,j)])

tabela_probabilidade['Rotas_Out'] = Rotas_Out
tabela_probabilidade['Rotas_In'] = Rotas_In
tabela_probabilidade['Distancia'] = Distancia

tabela_probabilidade['Tal'] = 1 / tabela_probabilidade['Distancia']


index = list(df.index)
index.remove(0)
valores = df.loc[index]['Val'] / df.loc[index]['Cyn']
maximo = max(valores)
feromonio_inicias = valores / (maximo)

for idx in index:
    mask = tabela_probabilidade['Rotas_In'] == idx
    tabela_probabilidade.loc[mask, 'Feromonio'] = feromonio_inicias[idx]
    
tabela_probabilidade['Feromonio'].fillna(0, inplace = True)

tabela_probabilidade['Tal_x_Feromonio'] = tabela_probabilidade['Tal'] * tabela_probabilidade['Feromonio']

#### Execucao

In [46]:
all_resultados = []
visitados = []
for ts in range(total_sellers):
    
    if ts > 0:
        resultados_, tabela_probabilidade = aco_execution_function(iteracoes, total_formigas, total_customers,tabela_probabilidade, df, change_duration, 
                                                service_duration, cost, velocity, truck_capacity, sellers_capacity, best, coef_evaporacao, 
                                                const_atualizacao_feromonio, sellers = True, rotas = visitados)
        all_resultados.append(resultados_)
        visitados = visitados + resultados_['rota'].copy()
        visitados.remove(0)
        visitados.remove(0)
        
    else:
        resultados_, tabela_probabilidade = aco_execution_function(iteracoes, total_formigas, total_customers,tabela_probabilidade, df, change_duration, 
                                                        service_duration, cost, velocity, truck_capacity, sellers_capacity, best, coef_evaporacao, 
                                                        const_atualizacao_feromonio)
        all_resultados.append(resultados_)
        visitados = visitados + resultados_['rota'].copy()
        visitados.remove(0)
        visitados.remove(0)